<a href="https://colab.research.google.com/github/mechanic3000/GB_NN_Intro/blob/Lesson_04/Lesson_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Практическое задание

<ol>
    <li>Попробовать улучшить точность распознования образов cifar 10 сверточной нейронной сетью, рассмотренной на уроке. Приложить анализ с описанием того, что улучшает работу нейронной сети и что ухудшает.
    </li>
    <li>Описать также в анализе какие необоходимо внести изменения  в получившуюся у вас нейронную сеть если бы ей нужно было работать не с cifar10, а с MNIST, CIFAR100 и IMAGENET.
    </li>
</ol>

In [4]:
from __future__ import print_function
import keras # расскоментируйте эту строку, чтобы начать обучение
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

# установка параметров нейросети
# batch_size = 32
# batch_size = 16
batch_size = 64
num_classes = 10
# epochs = 1
epochs = 10
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# разделение тренировочной и тестовой выборки
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'тренировочные примеры')
print(x_test.shape[0], 'тестовые примеры')

# преобразование матрицы чисел 0-9 в бинарную матрицу чисел 0-1
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# конфигурирование слоев нейросети
model = Sequential()

# слои нейросети отвественные за свертку и max-pooling
model.add(Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.15))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.15))

# полносвязные слои нейронной сети
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# инициализация RMSprop optimizer
opt = keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)

# компиляция модели
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not data_augmentation:
    print('Не используется data augmentation')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Использование data augmentation в реальном времени')
    # Препроцессинг и data augmentation в реальном времени:
    datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False, 
        zca_epsilon=1e-06, 
        rotation_range=0, 
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0., 
        zoom_range=0., 
        channel_shift_range=0.,
        fill_mode='nearest',
        cval=0.,
        horizontal_flip=True,
        vertical_flip=False,
        rescale=None,
        preprocessing_function=None,
        data_format=None,
        validation_split=0.0)

    # запуск data augmentation через fit
    # datagen.fit(x_train)

    # запуск data augmentation через fit_generator
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)

# сохранение модели и весов
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('сохранить обученную модель как %s ' % model_path)

# проверка работы обученной модели
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

x_train shape: (50000, 32, 32, 3)
50000 тренировочные примеры
10000 тестовые примеры
Использование data augmentation в реальном времени
Epoch 1/10


<ipython-input-4-c41aa95c2f64>:107: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(datagen.flow(x_train, y_train,


782/782 [==============================] - 36s 45ms/step - loss: 1.9939 - accuracy: 0.2695 - val_loss: 1.7090 - val_accuracy: 0.4004
Epoch 2/10
782/782 [==============================] - 35s 45ms/step - loss: 1.7393 - accuracy: 0.3674 - val_loss: 1.5614 - val_accuracy: 0.4380
Epoch 3/10
782/782 [==============================] - 36s 45ms/step - loss: 1.6232 - accuracy: 0.4077 - val_loss: 1.4870 - val_accuracy: 0.4698
Epoch 4/10
782/782 [==============================] - 35s 44ms/step - loss: 1.5438 - accuracy: 0.4350 - val_loss: 1.3666 - val_accuracy: 0.5043
Epoch 5/10
782/782 [==============================] - 34s 44ms/step - loss: 1.4850 - accuracy: 0.4630 - val_loss: 1.3350 - val_accuracy: 0.5195
Epoch 6/10
782/782 [==============================] - 36s 46ms/step - loss: 1.4334 - accuracy: 0.4838 - val_loss: 1.2805 - val_accuracy: 0.5469
Epoch 7/10
782/782 [==============================] - 35s 45ms/step - loss: 1.3884 - accuracy: 0.5027 - val_loss: 1.2585 - val_accuracy: 0.5515
Epo

In [ ]:
# Оценка исходной модели

# Test loss: 1.5661115646362305
# Test accuracy: 0.42969998717308044
# ###################################

# Оценка при изменении:

# Кол-во эпох = 3
# Test loss: 1.3396382331848145
# Test accuracy: 0.5149000287055969

# batch = 16
# Test loss: 1.1925313472747803
# Test accuracy: 0.5727999806404114

# значения Droput (0.1, 0.1, 0.15):
# Test loss: 1.1160888671875
# Test accuracy: 0.6051999926567078

# уменьшил количество слоев, убрал один pooling
# Test loss: 1.0581334829330444
# Test accuracy: 0.6236000061035156



# Выводы: 
# 1
# На увеличение оценки вляет увеличение количества эпох, 
#   уменьшение размера батча,
#   сничение количества данных попадающих в Дропаут (для частных случаев),
#   снижение количества слоев также сыграло положительную роль в увеличении оценки

# 2
  # Для работы с MNIST, CIFAR100 и IMAGENET в текущей сети необходимо изменить входные параметры,
  # т.к. у этих датасетов изображения разного размера, а также установить количество нейронов в последнем
  # слое соответствующее количеству классов в датасете.
  # Остальные параметры подбираются в процессе.